In [ ]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("/home/davis/deploy/davis/rm_isaac_bridge-pkg")
os.getcwd()

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import json
import numpy as np
import time
import threading
from collections import deque  

import io, ipywidgets, IPython
from PIL import Image
import matplotlib.pyplot as plt

from engine.pyalice import Application, Codelet
from engine.pyalice.gui.composite_widget import CompositeWidget
from engine.pyalice import Message, Composite

np.set_printoptions(precision=3)
cmap = plt.get_cmap('jet')
image_widget = ipywidgets.Image(height=720, width=1280)

In [ ]:
class CompositeArray(object):
    __slots__ = ('_measure', '_entities', '_quantities', '_values')
    def __init__(self, entities, measure, values):
        self._measure = measure
        self._entities = entities
        self._quantities = [[x, self._measure, 1] for x in self._entities]
        
        if type(values) != np.ndarray:
            values = np.array(values, dtype=np.float64)
        self._values = values
        
    @property
    def composite(self):
        return Composite.create_composite_message(self._quantities, self._values)
    
    @composite.setter
    def composite(self, composite):
        values = Composite.parse_composite_message(composite, self._quantities)
        if len(self._values) != len(values):
            raise ValueError('Size of state does not match number of values')
        self._values = values
        
    @property
    def values(self):
        return self._values
    
    @values.setter
    def values(self, values):
        if type(values) != np.ndarray:
            values = np.array(values, dtype=np.float64)
        self._values = values
        
class CompositeValue(object):
    __slots__ = ('_measure', '_entity', '_quantity', '_value')
    def __init__(self, entity, measure, value):
        self._measure = measure
        self._entity = entity
        self._quantity = [[self._entity, self._measure, 1]]
        
        self._value = np.array(value, dtype=np.float64)
        
    @property
    def composite(self):
        return Composite.create_composite_message(self._quantity, self._value)
    
    @composite.setter
    def composite(self, composite):
        value = Composite.parse_composite_message(composite, self._quantity)
        if len(values) != 1:
            raise ValueError('Size of state does not match number of values')
        self._value = value
        
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        value = np.array(value, dtype=np.float64)
        self._value = value

In [ ]:
class Command(object):
    __slots__ = ('action', 'payload', 'response')
    def __init__(self, action, payload):
        self.action = action
        self.payload = payload
        self.response = None

In [ ]:
class IsaacArm:
    def __init__(self, config):
        self.config = config
        self._timeout = config['timeout']
        
        self._control_panel = self.config['enable_control_panel']
        self._stream_joints = self.config['stream_joints']
        
        self.joint_names = self._load_kinematics(config['arm_type'])
        self.joints = CompositeArray(self.joint_names, 'position', [0]*len(self.joint_names))

        if self.config['enable_control_panel']:
            self.joint_widget = CompositeWidget(self.joint_names, 'position', [[-2*np.pi, 2*np.pi]] * len(self.joint_names))
        
        self._command_queue = deque()
        
        
    def _load_kinematics(self, arm_type):
        valid_kinematics = ['ur10', 'kinova_j2n7']
        if arm_type not in valid_kinematics:
            raise ValueError('No valid kinematic file found for '+arm_name+'. Valid kinematics exist for '+', '.join(valid_kinematics))
            
        self._kinematic_file = "apps/assets/kinematic_trees/{}.kinematic.json".format(arm_type)
        joint_names = []
        with open(self._kinematic_file,'r') as fd:
            kt = json.load(fd)
            for link in kt['links']:
                if 'motor' in link and link['motor']['type'] != 'constant':
                    joint_names.append(link['name'])
                    
        return joint_names
    
    def command(self, action, payload=None):
        if action not in ['set_joint_angles', 'get_joint_angles']:
            raise ValueError(action+' is not a valid action type')
        
        if type(payload) in [list, np.ndarray]:
            payload = CompositeArray(self.joint_names, 'position', payload)
            
        command = Command(action, payload)
        self._command_queue.append(command)
        
        elapsed = 0
        while command.response is None and elapsed < self._timeout:
            elapsed += 0.01
            time.sleep(0.01)
            
        return command.response
        
    def enable_joint_stream(self):
        self._stream_joints = True
    def disable_joint_stream(self):
        self._stream_joints = False
    
    def enable_all_streams(self):
        self._stream_joints = True
    def disable_all_streams(self):
        self._stream_joints = False
        
    def _JointReciever(self):
        parent = self
        class JointReciever(Codelet):
            def start(self):
                self.rx = self.isaac_proto_rx("CompositeProto", "state")
                self.tick_on_message(self.rx)
            
            def tick(self):
                if len(parent._command_queue) > 0 and parent._command_queue[0].action == 'get_joint_angles':
                    msg = self.rx.message
                    parent.joints.composite = msg
                    command = parent._command_queue.popleft()
                    command.response = parent.joints.values
                    
                elif parent._stream_joints:
                    msg = self.rx.message
                    parent.joints.composite = msg
                    
                else:
                    return
                
                if parent._control_panel:
                    parent.joint_widget.composite = msg
                    
                    
        return JointReciever
    
    def _JointTransmitter(self):
        parent = self
        class JointTransmitter(Codelet):
            def start(self):
                self.tx = self.isaac_proto_tx("CompositeProto", "command")
                self.tick_periodically(0.1)
                
            def tick(self):
                if len(parent._command_queue) > 0 and parent._command_queue[0].action == 'set_joint_angles':
                    self.tx._msg = parent._command_queue[0].payload.composite
                    self.tx.publish()
                    command = parent._command_queue.popleft()
                    command.response = True
                
                elif parent._control_panel and parent._stream_joints:
                    self.tx._msg = parent.joint_widget.composite
                    self.tx.publish()
                    
        return JointTransmitter
        
    def connect_app(self, app, config):
        # load dependency subgraphs
        app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
        simulation_interface = app.nodes["simulation.interface"]
        lqr_interface = app.nodes["lqr.subgraph"]["interface"]
        
        # configs
        app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = self._kinematic_file
        lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
        lqr_planner.config.speed_min = [-1.0] * len(self.joint_names)
        lqr_planner.config.speed_max = [1.0] * len(self.joint_names)
        lqr_planner.config.acceleration_min = [-1.0] * len(self.joint_names)
        lqr_planner.config.acceleration_max = [1.0] * len(self.joint_names)
        
        # create nodes
        joints_in_node = app.add("joints_input")
        joints_in_node.add(self._JointReciever(), 'joints_reciever')
        
        joints_out_node = app.add("joints_output")
        joints_out_node.add(self._JointTransmitter(), 'joints_transmitter')
                
        # connect edges
        app.connect(simulation_interface["output"], "joint_state", lqr_interface, "joint_state")
        app.connect(simulation_interface["output"], "joint_state", joints_in_node['joints_reciever'], "state")
        
        app.connect(joints_out_node['joints_transmitter'], "command", lqr_interface, "joint_target")
        app.connect(lqr_interface, "joint_command", simulation_interface["input"], "joint_position")
        
        return app

In [ ]:
class IsaacCamera(object):
    def __init__(self, config):
        self.config = config
        self._timeout = config['timeout']
        
        self.color = np.zeros((720,1280))
        self.depth = np.zeros((720,1280))
        
        self._control_panel = self.config['enable_control_panel']
        self._stream_color = self.config['stream_color']
        self._stream_depth = self.config['stream_depth']
        
        if self._control_panel:
            self.color_widget = ipywidgets.Image(height=310, width=640)
            self.depth_widget = ipywidgets.Image(height=310, width=640)
        
        self._command_queue = deque()
        
    def command(self, action, payload=None):
        if action not in ['get_depth', 'get_color']:
            raise ValueError(action+' is not a valid action type')

        command = Command(action, payload)
        self._command_queue.append(command)
        
        elapsed = 0
        while command.response is None and elapsed < self._timeout:
            elapsed += 0.01
            time.sleep(0.01)
            
        return command.response
    
    def enable_color_stream(self):
        self._stream_color = True
    def disable_color_stream(self):
        self._stream_color = False
        
    def enable_depth_stream(self):
        self._stream_depth = True
    def disable_depth_stream(self):
        self._stream_depth = False
        
    def enable_all_streams(self):
        self._stream_color = True
        self._stream_depth = True
    def disable_all_streams(self):
        self._stream_color = False
        self._stream_depth = False
        
    def _ColorReciever(self):
        parent = self
        class ColorReciever(Codelet):
            def start(self):
                self.rx = self.isaac_proto_rx("ColorCameraProto", "color")
                self.tick_on_message(self.rx)
            
            def tick(self):
                if len(parent._command_queue) > 0 and parent._command_queue[0].action == 'get_color':
                    msg = self.rx.message
                    tensor = msg.tensor
                    test_img = tensor
                    parent.color = tensor
                    command = parent._command_queue.popleft()
                    command.response = parent.color
                    
                elif parent._stream_color:
                    msg = self.rx.message
                    tensor = msg.tensor
                    parent.color = tensor
                    
                else:
                    return
                
                if parent._control_panel:
                    bts = io.BytesIO()
                    Image.fromarray(parent.color).save(bts, 'png')
                    parent.color_widget.value = bts.getvalue()
                        
                    
        return ColorReciever
    
    def _DepthReciever(self):
        parent = self
        class DepthReciever(Codelet):
            def start(self):
                self.rx = self.isaac_proto_rx("ColorCameraProto", "depth")
                self.tick_on_message(self.rx)
            
            def tick(self):
                if len(parent._command_queue) > 0 and parent._command_queue[0].action == 'get_depth':
                    msg = self.rx.message
                    tensor = msg.tensor
                    test_img = tensor
                    parent.depth = tensor
                    command = parent._command_queue.popleft()
                    command.response = parent.depth
                    
                elif parent._stream_depth:
                    msg = self.rx.message
                    tensor = msg.tensor
                    parent.depth = tensor
                    
                else:
                    return
                
                if parent._control_panel:
                    min_depth = np.min(parent.depth)
                    max_depth = np.max(parent.depth)
                    
                    normalized_1channel = np.array(255*(parent.depth-min_depth)/(max_depth-min_depth), dtype=np.uint8)
                    rgba_depth = np.array(255*cmap(normalized_1channel), dtype=np.uint8)
                    
                    bts = io.BytesIO()
                    Image.fromarray(rgba_depth).save(bts, 'png')
                    parent.depth_widget.value = bts.getvalue()
                    
        return DepthReciever
    
    def connect_app(self, app, config):
        
        simulation_interface = app.nodes["simulation.interface"]

        color_in_node = app.add('color_input')
        color_in_node.add(self._ColorReciever(), 'color_reciever')
        app.connect(simulation_interface["output"], 'color', color_in_node['color_reciever'], 'color')
        
        depth_in_node = app.add('depth_input')
        depth_in_node.add(self._DepthReciever(), 'depth_reciever')
        app.connect(simulation_interface["output"], 'depth', depth_in_node['depth_reciever'], 'depth')
        
        return app

In [ ]:
class RMIsaacBridge(object):
    def __init__(self, isaac_config):
        self.config = isaac_config
        
        self.arm = IsaacArm(isaac_config)
        self.camera = IsaacCamera(isaac_config)
        
        self._app = self._init_app(isaac_config)
        self._app = self.arm.connect_app(self._app, isaac_config)
        self._app = self.camera.connect_app(self._app, isaac_config)
        
        if self.config['enable_control_panel']:
            camera_widget = widgets.HBox([self.camera.color_widget, self.camera.depth_widget])
            arm_widget = self.arm.joint_widget.panel
            display(widgets.VBox([camera_widget, arm_widget]))

    def _init_app(self, isaac_config):
        app = Application(name="rm_isaac_bridge")
        app.load(filename="packages/navsim/apps/navsim_tcp.subgraph.json", prefix="simulation")
        return app
    
    def start_app(self):
        self._app.start()
        
    def stop_app(self):
        self._app.stop()

In [ ]:
isaac_config = {
    'arm_type': 'ur10',
    'enable_control_panel': True,
    'stream_depth': True,
    'stream_color': True,
    'stream_joints': True,
    'timeout': 0.3
}

rm = RMIsaacBridge(isaac_config)

In [ ]:
rm.start_app()

In [ ]:
rm.camera.disable_all_streams()

In [ ]:
rm.arm.disable_all_streams()

In [ ]:
rm.camera.command('get_depth')

In [ ]:
rm.camera.command('get_color')

In [ ]:
rm.arm.command('set_joint_angles', np.subtract(np.random.rand(6), 0.5))

In [ ]:
rm.arm.command('get_joint_angles')

In [ ]:
rm.stop_app()

In [ ]:
from ikpy.chain import Chain